# ゲート付きRNN

RNN層にゲートと呼ばれる機構を追加して長期的な文脈が保持できるようになったもの。

In [1]:
import random

import sentencepiece as spm
import torch
from torch import nn, optim
import torch.nn.functional as F
from torch.utils.data import Dataset, DataLoader, random_split
from torch.nn.utils.rnn import pad_sequence
from dlprog import train_progress

In [2]:
prog = train_progress(with_test=True)
device = torch.device('cuda' if torch.cuda.is_available() else 'cpu')
device

device(type='cuda')

### 学習データ

In [3]:
textfile = 'data/jawiki_2000.txt'
with open(textfile) as f:
    data = f.readlines()

In [5]:
tokenizer_prefix = 'models/tokenizer_jawiki_2000'
sp = spm.SentencePieceProcessor(f'{tokenizer_prefix}.model')
data_ids = sp.encode(data)
n_vocab = len(sp)

unk_id = sp.unk_id()
bos_id = sp.bos_id()
eos_id = sp.eos_id()
pad_id = sp.pad_id()

for ids in data_ids:
    if ids:
        ids[0] = bos_id
        ids[-1] = eos_id

In [6]:
class TextDataset(Dataset):
    def __init__(self, data_ids):
        self._n_samples = len(data_ids)
        self.data = [torch.tensor(ids) for ids in data_ids]

    def __getitem__(self, idx):
        in_text = self.data[idx][:-1]
        out_text = self.data[idx][1:]
        return in_text, out_text

    def __len__(self):
        return self._n_samples

def collate_fn(batch):
    in_text, out_text = zip(*batch)
    in_text = pad_sequence(in_text, batch_first=True, padding_value=pad_id)
    out_text = pad_sequence(out_text, batch_first=True, padding_value=pad_id)
    return in_text, out_text

batch_size = 32
dataset = TextDataset(data_ids)
train_dataset, test_dataset = random_split(dataset, [0.8, 0.2])
train_loader = DataLoader(
    train_dataset,
    batch_size=batch_size,
    shuffle=True,
    drop_last=True,
    collate_fn=collate_fn
)
test_loader = DataLoader(
    test_dataset,
    batch_size=batch_size,
    drop_last=True,
    collate_fn=collate_fn
)

sample = next(iter(train_loader))
sample[0].shape

torch.Size([32, 1433])


---

## ゲート

あるデータをどれくらい通すかを示したもの。具体的には対称のデータと同じサイズのベクトル。0-1の値をとる。  
NNで実装してみる。

In [7]:
class Gate(nn.Module):
    def __init__(self, input_size):
        super().__init__()
        self.net = nn.Sequential(
            nn.Linear(input_size, input_size),
            nn.Sigmoid()
        )

    def forward(self, x):
        return self.net(x)

入力されたデータを線形変化し、sigmoid関数に入力するだけ。このモデルにあるデータを入力したときの出力値が、そのデータのゲートとなる。  
ゲートを元のデータに掛けることで、元のデータの一部を**通した**ということになる。

In [8]:
input_size = 3
gate = Gate(input_size)

x = torch.randn(input_size)
y = x * gate(x)
print('input:', x)
print('gate:', gate(x))
print('output:', y)

input: tensor([ 0.3929, -0.7335, -1.4230])
gate: tensor([0.4253, 0.2988, 0.3143], grad_fn=<SigmoidBackward0>)
output: tensor([ 0.1671, -0.2192, -0.4472], grad_fn=<MulBackward0>)



---

## GRU

*Gate Recurrent Unit*

ゲート付きRNNの一種。

一旦RNNの復習をしよう。


RNNはある時間$t$の入力$x_t$に対して以下のような演算で出力値$h_t$を決定する。

$$
h_t = \mathrm{tanh}(W_x x_t + b_x + W_h h_{t-1} + b_h)
$$

この$x_t$と$h_{t-1}$の全結合の部分は$\mathrm{fc}(x,h)$で表すことにしよう。

$$
\begin{align}
h_t &= \mathrm{tanh}(\mathrm{fc}(x_t,h_{t-1})) \\
\mathrm{fc}(x,h) &= W_x x + b_x + W_h h + b_h
\end{align}
$$

んで、$\mathrm{fc}(x,h)$の実装もしておこう。

In [9]:
class FullyConnected(nn.Module):
    def __init__(self, input_size, hidden_size):
        super().__init__()
        self.fc_input = nn.Linear(input_size, hidden_size)
        self.fc_hidden = nn.Linear(hidden_size, hidden_size)

    def forward(self, x, h):
        return self.fc_input(x) + self.fc_hidden(h)

では、GRUの構造を見ていこう。  
GRUは以下のような演算で出力値$h_t$を決定する。

$$
\begin{align}
h_t &= (1 - z_t) \odot \tilde{h}_t + z_t \odot h_{t-1} \\
\tilde{h}_t &= \mathrm{tanh}(\mathrm{fc}_{\tilde h}(x_t,h_{t-1})) \\
z_t &= \sigma(\mathrm{fc}_{z}(x_t,h_{t-1})) \\
\end{align}
$$

$\sigma(x)$はsigmoid関数。

RNNでは新たなデータ$\tilde h_t$がそのまま出力されていた。  
GRUでは、新たなデータ$\tilde h_t$を古いデータ$h_{t-1}$に足して出力する。そして、その際の比率をゲート$z_t$で決める。この$z_t$は$h_{t-1}$をどれだけ通すかを表す。

In [10]:
class SimpleGRU(nn.Module):
    def __init__(self, input_size, hidden_size):
        super().__init__()
        self.fc = FullyConnected(input_size, hidden_size)
        self.gate = nn.Sequential(
            FullyConnected(input_size, hidden_size),
            nn.Sigmoid()
        )

    def forward(self, x, h):
        h_new = F.tanh(self.fc(x, h))
        z = self.gate(x)
        h = (1 - z) * h_new + z * h
        return h

このように、GRUではゲートを用いて新たなデータをどれだけ取り入れるべきか、そして古いデータをどれだけ捨てるか考えることが出来る。  
この枠組みの下で学習を行うことで、長期的に保持すべきデータをしっかりと保持できるようになることが期待される。

ちなみに、上記のモデルは一般的なGRUを私が簡略化したもの。  
一般的なGRUは、上記のモデルにゲートを一つ追加した以下のモデルである。


$$
\begin{align}
h_t &= (1 - z_t) \odot \tilde{h}_t + z_t \odot h_{t-1} \\
\tilde{h}_t &= \mathrm{tanh}(\mathrm{fc}_{\tilde h}(x_t,r_t \odot h_{t-1})) \\
z_t &= \sigma(\mathrm{fc}_{z}(x_t,h_{t-1})) \\
r_t &= \sigma(\mathrm{fc}_{r}(x_t,h_{t-1})) \\
\end{align}
$$

新なデータ$\tilde h_t$を生成する際に、古いデータ$h_{t-1}$をどれだけ考慮するかを決めるゲート$r_t$が追加されている。

In [11]:
class GRU(nn.Module):
    def __init__(self, input_size, hidden_size):
        super().__init__()
        self.fc_input = FullyConnected(input_size, hidden_size)
        self.gate_update = nn.Sequential(
            FullyConnected(input_size, hidden_size),
            nn.Sigmoid()
        )
        self.gate_reset = nn.Sequential(
            FullyConnected(input_size, hidden_size),
            nn.Sigmoid()
        )

    def forward(self, x, h):
        r = self.gate_reset(x, h)
        h_new = F.tanh(self.fc_input(x, r * h))
        z = self.gate_update(x)
        h = (1 - z) * h_new + z * h
        return h

また、RNN同様、PyTorchにクラスとして`torch.nn.GRU`が用意されている:  
[GRU — PyTorch 2.0 documentation](https://pytorch.org/docs/stable/generated/torch.nn.GRU.html)

In [12]:
gru = nn.GRU(input_size, input_size)


---

## LSTM

*Long Short-Term Memory*

長短期記憶

GRUの進化版。考え方はGRUと同じで、RNNにゲートを取り入れてイイ感じにしたもの。  
ちなみに、GRUよりLSTMの方が先に提案されている。GRUはLSTMの簡易版として後から提案された。

LSTMには出力する隠れ状態$h_t$だけでなく、**記憶セル**と呼ばれる変数$c_t$を持つ。記憶セルはLSTMの外に出力されることはなく、LSTM内部でのみ使用される。

まず簡単に文字で説明する。  
記憶セル$c_t$がGRUでの隠れ状態$h_t$に当たり、ゲートを用いた不要な情報の削除と新たな情報の追加が行われる。なおゲートの生成には入力$x_t$と前の隠れ状態$h_{t-1}$を用いる（記憶セルは用いない）。そしてこの記憶セルを活性化関数に通したものをLSTMの出力=隠れ状態$h_t$とする。

具体的な構造を見てみよう。

$$
\begin{align}
h_t &= o_t \odot \mathrm{tanh}(c_t) \\
c_t &= f_t \odot c_{t-1} + i_t \odot \tilde c_t \\
\tilde c_t &= \mathrm{tanh}(\mathrm{fc}_{\tilde c}(x_t,h_{t-1})) \\
i_t &= \sigma(\mathrm{fc}_{i}(x_t,h_{t-1})) \\
f_t &= \sigma(\mathrm{fc}_{f}(x_t,h_{t-1})) \\
o_t &= \sigma(\mathrm{fc}_{o}(x_t,h_{t-1})) \\
\end{align}
$$


- $\tilde c_t$: 新たな情報。
- $i_t$: inputゲート。新たな情報$\tilde c_t$をどれだけ取り入れるかを決める。
- $f_t$: forgetゲート。古い情報$c_{h-1}$をどれだけ保持するかを決めるゲート。
- $o_t$: outputゲート。出力する隠れ状態の量を決めるゲート。

GRUでは1つのゲートを用いて新たな情報と古い情報の比率を決めていたが、LSTMでは別々のゲートを用いて決める。

実装は以下の通り。

In [13]:
class LSTM(nn.Module):
    def __init__(self, input_size, hidden_size):
        super().__init__()
        self.gate_input = nn.Sequential(
            FullyConnected(input_size, hidden_size),
            nn.Sigmoid()
        )
        self.gate_forget = nn.Sequential(
            FullyConnected(input_size, hidden_size),
            nn.Sigmoid()
        )
        self.gate_output = nn.Sequential(
            FullyConnected(input_size, hidden_size),
            nn.Sigmoid()
        )
        self.fc = FullyConnected(input_size, hidden_size)

    def forward(self, x, h, c):
        c_new = F.tanh(self.fc(x, h))
        i = self.gate_input(x, h)
        f = self.gate_forget(x, h)
        o = self.gate_output(x, h)
        c = f * c + i * c_new
        h = o * F.tanh(c)
        return h, c

PyTorchにも`torch.nn.LSTM`が用意されている:  
[LSTM — PyTorch 2.0 documentation](https://pytorch.org/docs/stable/generated/torch.nn.LSTM.html)


---

## LSTMを用いた言語モデル

せっかくなので、LSTMで言語モデルを作ってみよう。RNNLMのRNN層をLSTMに変更するだけ。

モデル。RNN層の部分をLSTMに変更する。  
LSTMは隠れ状態$h$と記憶セル$c$の2つを出力するので、それらを与える・受け取ることが出来るようにする。

In [14]:
class LanguageModel(nn.Module):
    def __init__(self, n_vocab, embed_size, hidden_size):
        super().__init__()
        self.embedding = nn.Embedding(n_vocab, embed_size)
        self.lstm = nn.LSTM(embed_size, hidden_size, batch_first=True)
        self.fc = nn.Linear(hidden_size, n_vocab)

    def forward(self, x, hc=None):
        x = self.embedding(x) # (seq_len, embed_size)
        y, (h, c) = self.lstm(x, hc) # (seq_len, hidden_size)
        y = self.fc(y) # (seq_len, n_vocab)
        return y, (h, c)

では学習させる。  
ミニバッチ&Truncated BPTT。

In [15]:
cross_entropy = nn.CrossEntropyLoss(ignore_index=pad_id) # padトークンを無視
def loss_fn(y, t):
    """
    y: (batch_size, seq_length, n_vocab)
    t: (batch_size, seq_length)
    """
    loss = cross_entropy(y.reshape(-1, n_vocab), t.ravel())
    return loss

def eval_model(model):
    model.eval()
    total_loss = 0
    with torch.no_grad():
        for x, t in test_loader:
            x = x.to(device)
            t = t.to(device)
            y, _ = model(x)
            loss = loss_fn(y, t)
            total_loss += loss.item()
    loss = total_loss / len(test_loader)
    return loss


def train(model, optimizer, trunc_len, n_epochs, prog_unit=1):
    prog.start(n_iter=len(train_loader), n_epochs=n_epochs, unit=prog_unit)
    for epoch in range(1, n_epochs + 1):
        model.train()
        for x, t in train_loader:
            hc = None
            x = x.to(device)
            t = t.to(device)
            for i in range(0, len(x), trunc_len):
                x_batch = x[:, i:i+trunc_len]
                t_batch = t[:, i:i+trunc_len]
                optimizer.zero_grad()
                y, (h, c) = model(x_batch, hc)
                loss = loss_fn(y, t_batch)
                loss.backward()
                optimizer.step()
                prog.update(loss.item(), advance=0)
                h = h.detach()
                c = c.detach()
                hc = (h, c)
            prog.update(advance=1)
        if epoch % prog_unit == 0:
            test_loss = eval_model(model)
            prog.memo(f'test: {test_loss:.5f}')
        else:
            prog.memo('')

In [18]:
hidden_size = 512
model = LanguageModel(n_vocab, hidden_size, hidden_size).to(device)
optimizer = optim.Adam(model.parameters(), lr=1e-4)

In [19]:
train(model, optimizer, 64, 1000, 100)

    1-100/1000: ############################## 100% [00:01:20.65] loss train: 5.29094, test: 7.10566 
  101-200/1000: ############################## 100% [00:01:26.00] loss train: 1.86016, test: 8.70382 
  201-300/1000: ############################## 100% [00:01:25.91] loss train: 0.45740, test: 10.13630 
  301-400/1000: ############################## 100% [00:01:26.05] loss train: 0.18387, test: 10.88216 
  401-500/1000: ############################## 100% [00:01:26.86] loss train: 0.14449, test: 11.68350 
  501-600/1000: ############################## 100% [00:01:27.43] loss train: 0.13665, test: 12.20779 
  601-700/1000: ############################## 100% [00:01:27.13] loss train: 0.13358, test: 12.63444 
  701-800/1000: ############################## 100% [00:01:25.18] loss train: 0.13228, test: 12.89107 
  801-900/1000: ############################## 100% [00:01:25.61] loss train: 0.13133, test: 13.10439 
 901-1000/1000: ############################## 100% [00:01:27.12] loss trai

相変わらずtestデータの誤差は減らない。

文章生成。

In [20]:
def token_sampling(y):
    y = y[-1]
    probs = F.softmax(y, dim=-1)
    token = random.choices(range(n_vocab), weights=probs)[0]
    return token


bos_id = sp.bos_id()
eos_id = sp.eos_id()
@torch.no_grad()
def generate_sentence(
    model: nn.Module,
    start: str = '',
    max_len: int = 50
) -> str:
    model.eval()
    start_ids = sp.encode(start) if start else [bos_id]
    start_ids = torch.tensor(start_ids, device=device)

    y, (h, c) = model(start_ids)
    next_token = token_sampling(y)
    ids = [next_token]
    for _ in range(max_len):
        x = torch.tensor([next_token], device=device)
        y, (h, c) = model(x, (h, c))
        next_token = token_sampling(y)
        ids.append(next_token)
        if next_token == eos_id:
            break
    sentence = start + sp.decode(ids)
    return sentence

In [21]:
for _ in range(10):
    print(generate_sentence(model, max_len=50))

親友だったピアニスト・作曲家のルートヴィヒ・シュンケに献呈された。シュンケは返礼として『大ソナタ ト短調』作品3を献呈したが、翌1834年に肺結核のため23歳で夭
1936年にイギリス空軍から出された双発中型爆撃機の仕様(P.13/36)に従って開発されたのがマンチェスターで、開発当初はアブロ679と呼ばれていた。本機の特徴は、当時開発中だった高出力の
1947年(昭和22年)4月の日本証券取引所解散をうけ、証券会社各社の出資で発足した。現在は、不動産開発、宅地分譲やマンション分譲、ショッピングセンターなどのデベロッパー
石川県金沢市生まれ。作家・杉森久英の長女。子ども時代にバレエを習い始め、のち日本舞踊を修める。1967年東京大学仏文科卒、1970年同大学院修士課程修了、博士課程
映画はアメリカ合衆国で2008年10月17日に公開された。 一般公開前にロイ・ディズニーが招待したヨット・レースファンのためにプライベート・スクリーンで、2008年3月14日にニューポート(ロードアイランド州)で試写
凄腕の傭兵、リロイ・シュヴァルツァーと、その相棒である意思を持ち、言葉を話す剣ラグナロクの活躍を描くアクションファンタジー。リアルなバトルを彷彿させる手法が特徴で、本編
ラグビー部は全国高等学校ラグビーフットボール大会出場経験あり。メカトロ部は全日本ロボット相撲大会において2011年(平成23年)大会まで3連覇を達成したが、2012年(平成24年)大会で
Microsoft Outlook、Windows Live Hotmail、Yahoo!メール、Gmail、iCloud、Exchangeなどに標準対応している。設定をすることでPOP
平地や山地の林に生息し、おもに木の実を食べる。地上2～6mの枝の上に、小枝、枯草、 ⁇ 類で皿形の巣を作る。5～7月に卵を
いつごろからか歴史の影に現れ始め、ついには人類を滅亡寸前まで追い詰めた異形の存在。総じて強靭な生命力を持ち、人類に対して敵意を持つ。その戦闘力、危険度から


マシになった気がする。